# Contexto analítico

Nesse desafio, você deve resolver um case de precificação de imóveis. Esse desafio foi construído em parceria entre a Tera e o QuintoAndar, onde o objetivo é simular um projeto de machine learning com características semelhantes ao que ocorre no dia a dia da empresa.

Imagine-se na seguinte situação: a área de marketing do QuintoAndar quer montar uma calculadora de preço (como esta [aqui: https://mkt.quintoandar.com.br/quanto-cobrar-de-aluguel/), e nesse projeto, os analistas negociais e corretores querem, também, entender as principais variáveis e características chaves que influenciam no valor de venda do imóvel (Ex: quantificar o impacto do aumento da área do imóvel no preço, ou quantificar o impacto de ter piscina, ou não no preço). Você é o cientista de dados que atuará na resolução desse case.

Para tanto, existem dois objetivos principais:

**Objetivo 1, interpretabilidade:** construir uma regressão linear simples, com poucas variáveis importantes, visando gerar insights para corretores e proprietários no quesito precificação dos imóveis. Ou seja, o foco será na interpretação dos coeficientes (ex: se aumentar a área do imóvel em uma unidade isso irá aumentar em Y o preço deste imóvel).

**Objetivo 2, predição:** construir um modelo com alto poder preditivo, com mais variáveis, visando um bom desempenho e com o intuito de ser usado em uma página web como a calculadora de preço. Note que, em uma situação real, um alto erro de inferência pode gerar grande insatisfação em um proprietário de imóvel, que pode ficar ofendido com o resultado. Por isso, em casos como esse, queremos ter o menor erro possível, mesmo que o modelo  seja complexo e tenha uma interpretação mais difícil.

<h1>Objetivo2:  regressão via Random Forest</h1>

In [124]:
import numpy as np
import seaborn as sns
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

In [125]:
path = '/Users/Akihiro/Desktop/TERA/Módulo_4/Desafio_QuintoAndar'
df = pd.read_csv(path+'/base.csv')
y = np.log(df['PrecoVenda'])
X = df.drop('PrecoVenda', axis=1)

<h2>Separar em treino, validação e teste</h2>

In [126]:
# Primeiro use a função função train_test_split  
# para separar 70% para treino e 30% para validação e teste.
X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.7, test_size=0.3, random_state=42)

# Segundo, aplique novamente essa função para quebrar esses 30% em dois datasets, 
# sendo 50% para teste e 50% para validação. 
# Assim obtendo 70% para treino, 15% para validação e 15% para teste.
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, train_size=0.5, test_size=0.5, random_state=42)

print(X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape)

(1021, 80) (219, 80) (219, 80) (1021,) (219,) (219,)


<h2>Separar variáveis numéricas e categóricas</h2>

In [127]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 80 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Id                    1460 non-null   int64  
 1   ClasseImovel          1460 non-null   int64  
 2   ClasseZona            1460 non-null   object 
 3   Fachada               1201 non-null   float64
 4   TamanhoLote           1460 non-null   int64  
 5   Rua                   1460 non-null   object 
 6   Beco                  91 non-null     object 
 7   FormaProp             1460 non-null   object 
 8   PlanoProp             1460 non-null   object 
 9   Servicos              1460 non-null   object 
 10  ConfigLote            1460 non-null   object 
 11  InclinacaoLote        1460 non-null   object 
 12  Bairro                1460 non-null   object 
 13  Estrada1              1460 non-null   object 
 14  Estrada2              1460 non-null   object 
 15  TipoHabitacao        

In [128]:
var_categorica = [col for col in X.copy() if X[col].dtype=='object'] 
var_numerica = [col for col in X.copy() if X[col].dtype=='float64' or X[col].dtype=='int64']

colunas = var_numerica + var_categorica
X_train = X_train[colunas].copy()
X_val = X_val[colunas].copy()

In [129]:
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((1021, 80), (219, 80), (1021,), (219,))

## Pré-processamento e baseline

In [130]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

# atualizar variáveis numéricas e categóricas, além de atualizar o X_train e X_val.
def train_test_split_update(df_X = X, 
                            df_X_train=X_train, 
                            df_X_val=X_val):
    var_categorica = [col for col in df_X.copy() if df_X[col].dtype=='object'] 
    var_numerica = [col for col in df_X.copy() if df_X[col].dtype=='float64' or df_X[col].dtype=='int64']
    colunas = var_numerica + var_categorica
    return  df_X_train[colunas].copy(), df_X_val[colunas].copy(), var_categorica, var_numerica

# atualizar variáveis numéricas e categóricas.
X_train, X_val, var_categorica, var_numerica = train_test_split_update(df_X = X, 
                                                                       df_X_train=X_train, 
                                                                       df_X_val=X_val)
# Pipeline do pré-processamento e modelo.
def pipeline_geral(variavel_numerica = var_numerica, 
                        variavel_categorica = var_categorica,
                        modelo = RandomForestRegressor(n_estimators=150, random_state=0)):
    # Pré-processamento para variável numérica:
    transformador_numerico = SimpleImputer(strategy='median')

    # Pré-processamento para variável categórica:
    transformador_categorico = Pipeline(steps=[('imputer',  SimpleImputer(strategy='most_frequent')), 
                                               ('onehot',  OneHotEncoder(handle_unknown='ignore'))])
    # Pré-processamento geral:
    transformador_geral = ColumnTransformer(
        transformers=[
            ('numerico', transformador_numerico, variavel_numerica),
            ('categorico', transformador_categorico, variavel_categorica)])
    
    # modelo baseline RandomForest
    modelo = modelo

    # Pré-processamento e modelo
    pipe_modelo = Pipeline(steps=[('transformador', transformador_geral),
                             ('modelo', modelo)])
    return pipe_modelo

# Pré-processamento e treino de modelo
pipe_modelo = pipeline_geral(variavel_numerica = var_numerica, 
                                  variavel_categorica = var_categorica,
                                  modelo = RandomForestRegressor(n_estimators=150, random_state=0))

modelo_fit = pipe_modelo.fit(X_train, y_train)
# Pré-processamento e validação de modelo
previsao = pipe_modelo.predict(X_val)

def results(X_train=X_train, col=X.columns, y_val=y_val, previsao=previsao):   
    n = len(X_train)
    p = len(col)
    R2 = r2_score(y_val, previsao)
    adj_R2 = 1-((1-R2) * (n-1)/(n-p-1)) #Adj R2 = 1-(1-R2)*(n-1)/(n-p-1)
    print('MAE: %.2f' % mean_absolute_error(np.exp(y_val), np.exp(previsao)), 'R2 adjusted: %.2f' % adj_R2)

results(X_train=X_train, col=X.columns, y_val=y_val, previsao=previsao)

MAE: 16130.29 R2 adjusted: 0.86


* A baseline tem um erro médio absoluto em **16 130** dólares.
* A baseline tem um erro r2 ajustado de 0.86.

## Importância de Feature

In [131]:
from sklearn.inspection import permutation_importance

In [132]:
def feature_imp(modelo=modelo_fit, 
                X_val=X_val, 
                y_val=y_val, 
                n_repeats=10, 
                random_state=0):
    
    result = permutation_importance(modelo_fit, 
                                    X_val, y_val, 
                                    n_repeats=10,
                                random_state=0)
    
    result_df = pd.DataFrame({'Correlação': result.importances_mean})
    feature_name = pd.DataFrame({'Features': X_val.columns})
    feature_importance = pd.concat([feature_name, result_df], join='inner', axis=1)
    return feature_importance.sort_values(by='Correlação', ascending=False)

In [133]:
var_importantes = feature_imp(modelo=modelo_fit, 
                              X_val=X_val, 
                              y_val=y_val, 
                              n_repeats=10, 
                              random_state=0)

In [134]:
var_40_importantes = var_importantes.iloc[:40]
var_40_importantes = var_40_importantes.Features.values
var_importantes.head(5)

,Features,Correlação
4,Qualidade,0.385420
16,AreaConstruida,0.115408
12,AreaPorao,0.019730
13,AreaTerreo,0.010264
9,AreaAcabPorao1,0.009867


### Treinar e validar o novo modelo

In [148]:
X_new = df[var_40_importantes].copy()
y_new = np.log(df['PrecoVenda'].copy())
X_train_new, X_val_new, y_train_new, y_val_new = train_test_split(X_new, 
                                                                  y_new, 
                                                                  train_size=.7, 
                                                                  test_size=.3, 
                                                                  random_state=42)

X_val_new, X_test_new, y_val_new, y_test_new = train_test_split(X_val_new, 
                                                                y_val_new, 
                                                                train_size=0.5, 
                                                                test_size=0.5, 
                                                                random_state=42)

X_train_new.shape, X_val_new.shape, y_train_new.shape, y_val_new.shape, X_test_new.shape  

((1021, 40), (219, 40), (1021,), (219,), (219, 40))

In [184]:
X_test_new.head(3)

,Qualidade,AreaConstruida,AreaPorao,AreaTerreo,AreaAcabPorao1,TamanhoLote,CarrosGaragem,AnoReforma,AnoConstrucao,Condicao,...,TipoTelhado,TipoAlvenaria,Banheiro,AreaVarandaFechada,AreaAlvenaria,Lavabo,Estrada1,MAE,RMSE,R2
1418,5,1144,1144,1144,25,9204,1,1963,1963,5,...,Gable,None,1,0,0.0,1,Norm,17249.698775,29570.299934,0.885103
44,5,1150,1150,1150,179,7945,1,1959,1959,6,...,Gable,None,1,0,0.0,0,Norm,17249.698775,29570.299934,0.885103
588,5,1473,1437,1473,1324,25095,1,2003,1968,8,...,Flat,None,1,0,0.0,0,Norm,17249.698775,29570.299934,0.885103


In [149]:
# Usar função train_test_split_update
X_train_new, X_val_new, var_cat_2, var_num_2 = train_test_split_update(df_X = X_new, 
                                                 df_X_train=X_train_new, 
                                                 df_X_val=X_val_new)

pipe_modelo_2 = pipeline_geral(variavel_numerica = var_num_2, 
                        variavel_categorica = var_cat_2,
                        modelo = RandomForestRegressor(n_estimators=150, random_state=0))

modelo_fit_2 = pipe_modelo_2.fit(X_train_new, y_train_new)

# Pré-processamento e validação de modelo
previsao_2 = modelo_fit_2.predict(X_val_new)

# Resultado
results(X_train=X_train_new, col=X_new.columns, y_val=y_val_new, previsao=previsao_2)

MAE: 15851.07 R2 adjusted: 0.88


* O Erro Médio Absoluto deu 15 851 dólares.
* O R2 ajustado de 0.88.

**Comentário sobre importância das features**
* A importância das features diminuiu o EMA em 300 dólares e o R2 ajustado aumentou em 0.02, o modelo processou o resultado mais rápido do que usar todas as features do dataset. Embora a importância das features seja um método mais rápido do que construir features, é apropriado retornar um estudo mais aprofundado sobre as variáveis influenciadoras no preço da casa. Novas features podem impactar postivamente o Erro Médio Absoluto e o R2, como a relação do m2 da cozinha pelo m2 do lote, além de diminuir a multicolinearidade, levando à diminuição do erro. Infelizmente, a importância de permutação pode criar uma forte multicolinearidade do modelo.

## Otimização dos hiperparâmetros

In [150]:
list_estimator = [200, 225, 250, 275, 300, 325, 350, 375, 400, 425, 450]
maxdepth = [10]
n_estimators = 150
def otimizador(max_leaf_nodes=list_estimator):
    for i in range(len(list_estimator)):
        
        pipe_modelo_2 = pipeline_geral(variavel_numerica = var_num_2, 
                                variavel_categorica = var_cat_2,
                                modelo = RandomForestRegressor(n_estimators=60, 
                                                               max_leaf_nodes=list_estimator[i],
                                                               max_depth=40,
                                                               random_state=0))

        modelo_fit_2 = pipe_modelo_2.fit(X_train_new, y_train_new)

        # Pré-processamento e validação de modelo
        previsao_2 = modelo_fit_2.predict(X_val_new)

        # Resultado
        print('Max_leaf_nodes', list_estimator[i], ':', results(X_train=X_train_new, 
                                                                col=X_new.columns, 
                                                                y_val=y_val_new, 
                                                                previsao=previsao_2))        

In [151]:
otimizador(list_estimator)

MAE: 15659.02 R2 adjusted: 0.88
Max_leaf_nodes 200 : None
MAE: 15647.27 R2 adjusted: 0.88
Max_leaf_nodes 225 : None
MAE: 15642.33 R2 adjusted: 0.88
Max_leaf_nodes 250 : None
MAE: 15649.32 R2 adjusted: 0.88
Max_leaf_nodes 275 : None
MAE: 15630.20 R2 adjusted: 0.88
Max_leaf_nodes 300 : None
MAE: 15592.09 R2 adjusted: 0.88
Max_leaf_nodes 325 : None
MAE: 15608.37 R2 adjusted: 0.88
Max_leaf_nodes 350 : None
MAE: 15594.78 R2 adjusted: 0.88
Max_leaf_nodes 375 : None
MAE: 15596.77 R2 adjusted: 0.88
Max_leaf_nodes 400 : None
MAE: 15596.34 R2 adjusted: 0.88
Max_leaf_nodes 425 : None
MAE: 15602.38 R2 adjusted: 0.88
Max_leaf_nodes 450 : None


* Erro Médio Absoluto: 15 679 dólares
    * Max_leaf_nodes 425 
    * N_estimators 150
    
* Erro Médio Absoluto: 15 654 dólares
    * Max_leaf_nodes 375 
    * N_estimators 80

* Erro Médio Absoluto: 15 617 dólares
    * Max_leaf_nodes 325 
    * N_estimators 70

* Erro Médio Absoluto: 15 594 dólares
    * Max_leaf_nodes 375 
    * N_estimators 60
    
**O menor erro médio foi 15 594 dólares com os respectivos parâmetros:**
* n_estimators = 60
* max_leaf_nodes = 375

### Hiperparametrizar o modelo

In [152]:
pipe_modelo_2 = pipeline_geral(variavel_numerica = var_num_2, 
                        variavel_categorica = var_cat_2,
                        modelo = RandomForestRegressor(n_estimators=60, 
                                                       max_leaf_nodes=375,
                                                       random_state=0))

modelo_fit_2 = pipe_modelo_2.fit(X_train_new, y_train_new)

# Pré-processamento e validação de modelo
previsao_2 = modelo_fit_2.predict(X_val_new)

# Resultado
results(X_train=X_train_new, col=X_new.columns, y_val=y_val_new, previsao=previsao_2)

MAE: 15594.78 R2 adjusted: 0.88


# Dataset de teste

* Finalmente, compute as métricas de avaliação no dataset de teste para obter o proxy de performance do seu modelo em um ambiente em produção (ambiente real online).

In [153]:
def R2_adj(X_test=X_test_new, col=X.columns, y_test=y_test_new, previsao=previsao_2):   
    n = len(X_test_new)
    p = len(col)
    R2 = r2_score(y_test_new, previsao_2)
    adj_R2 = 1-((1-R2) * (n-1)/(n-p-1)) #Adj R2 = 1-(1-R2)*(n-1)/(n-p-1)
    return '%.2f' % adj_R2

**Reordenar as colunas de acordo com o X_val.**

In [178]:
X_test_new.reindex(columns=X_val_new.columns.values).head()

,Qualidade,AreaConstruida,AreaPorao,AreaTerreo,AreaAcabPorao1,TamanhoLote,CarrosGaragem,AnoReforma,AnoConstrucao,Condicao,...,InstalacaoEletrica,ParedePorao,QualidadeCozinha,Bairro,TipoAcabPorao1,Exterior2,PlanoProp,TipoTelhado,TipoAlvenaria,Estrada1
1418,5,1144,1144,1144,25,9204,1,1963,1963,5,...,SBrkr,No,TA,NAmes,BLQ,HdBoard,Lvl,Gable,None,Norm
44,5,1150,1150,1150,179,7945,1,1959,1959,6,...,FuseA,No,TA,NAmes,ALQ,Wd Sdng,Lvl,Gable,None,Norm
588,5,1473,1437,1473,1324,25095,1,2003,1968,8,...,SBrkr,Gd,Ex,ClearCr,GLQ,Wd Sdng,Low,Flat,None,Norm
1048,5,1771,0,1771,0,21750,2,2006,1960,4,...,SBrkr,NaN,TA,Mitchel,NaN,HdBoard,Lvl,Hip,BrkFace,Norm
439,6,1196,684,684,0,12354,2,2000,1920,8,...,SBrkr,Mn,Gd,Edwards,Unf,Wd Sdng,Lvl,Gable,None,Norm


**Modelo em dataset de teste:**

In [179]:
previsao_2 = modelo_fit_2.predict(X_test_new.reindex(columns=X_val_new.columns.values))

**Métricas computadas: MAE, RMSE, R2**

In [180]:
X_test_new['MAE'] = mean_absolute_error(np.exp(y_test_new), np.exp(previsao_2))
X_test_new['RMSE'] = (np.square(np.subtract(np.exp(y_test_new), np.exp(previsao_2))).mean())**.5
X_test_new['R2'] = r2_score(np.exp(y_test_new), np.exp(previsao_2))

In [183]:
X_test_new.head(3)

,Qualidade,AreaConstruida,AreaPorao,AreaTerreo,AreaAcabPorao1,TamanhoLote,CarrosGaragem,AnoReforma,AnoConstrucao,Condicao,...,TipoTelhado,TipoAlvenaria,Banheiro,AreaVarandaFechada,AreaAlvenaria,Lavabo,Estrada1,MAE,RMSE,R2
1418,5,1144,1144,1144,25,9204,1,1963,1963,5,...,Gable,None,1,0,0.0,1,Norm,17249.698775,29570.299934,0.885103
44,5,1150,1150,1150,179,7945,1,1959,1959,6,...,Gable,None,1,0,0.0,0,Norm,17249.698775,29570.299934,0.885103
588,5,1473,1437,1473,1324,25095,1,2003,1968,8,...,Flat,None,1,0,0.0,0,Norm,17249.698775,29570.299934,0.885103


### Conclusão:

* 